<a href="https://colab.research.google.com/github/cosminnedescu/ProjectMLDL/blob/main/owr/owr-ensebles-v2.0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Avoid K80
!nvidia-smi

Mon Jul 12 19:30:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import random

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.autograd import Variable

import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import Subset, DataLoader

from PIL import Image

import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from copy import copy
from copy import deepcopy

#### Cloning the Git repository

In [ ]:
!rm -rf ProjectMLDL
if not os.path.isdir('/content/ProjectMLDL'):
  !git clone https://github.com/cosminnedescu/ProjectMLDL.git
  %cd /content/ProjectMLDL
  !rm -rf LICENSE README.md

Cloning into 'ProjectMLDL'...
remote: Enumerating objects: 3379, done.
remote: Counting objects: 100% (795/795), done.
remote: Compressing objects: 100% (475/475), done.
remote: Total 3379 (delta 470), reused 537 (delta 307), pack-reused 2584
Receiving objects: 100% (3379/3379), 170.43 MiB | 29.63 MiB/s, done.
Resolving deltas: 100% (1855/1855), done.
Checking out files: 100% (359/359), done.
/content/ProjectMLDL


In [ ]:
from data.cifar100 import CIFAR100
from model.resnet32_modified import resnet32
import data.utils
from model.owrEnsamble import owrEnsemble

In [ ]:
# True mean and std of Cifar100 dataset (src="https://gist.github.com/weiaicunzai/e623931921efefd4c331622c344d8151")
mean = [0.5071, 0.4867, 0.4408]
std = [0.2675, 0.2565, 0.2761]

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize(mean, std),
     ])

## Incremental Classifier and Representation Learning

### Defining hyperparameters according to iCarl paper

In [ ]:
# Settings
DEVICE = 'cuda'
NUM_CLASSES = 100         # Total number of classes
VAL_SIZE = 0.2            # Proportion of validation set with respect to training set (between 0 and 1)

# Training
BATCH_SIZE = 128          # Batch size
LR = 0.1                    # Initial learning rate
                       
MOMENTUM = 0.9            # Momentum for stochastic gradient descent (SGD)
WEIGHT_DECAY = 1e-5       # Weight decay from iCaRL

RANDOM_SEED = [20]  # Random seeds defining the runs of every method
                          # Note: this should be at least 3 to have a fair benchmark

NUM_EPOCHS = 70           # Total number of training epochs
MILESTONES = [43, 63]     # Step down policy from iCaRL (MultiStepLR)
                          # Decrease the learning rate by gamma at each milestone
GAMMA = 0.2               # Gamma factor from iCaRL (1/5)

HERDING = False           # True to perform prioritized selection, False to perform random selection
CLASSIFY = True           # True to use mean-of-exemplar classifier, False to use network's output directly for classification

REJECTION_THRESHOLD = [0.50,0.75,0.95]
TESTING_MODE = 'harmonic' #open, closed, harmonic
N_ESTIMATORS = 5
CONFIDENCE = '0.900' #'0.900':1.47, '0.950':1.94, '0.975':2.45, '0.990':3.36
    

In [ ]:
def do_group_classes(run):

  train_subset = [[] for i in range(10)]
  train_dataloader = [[] for i in range(10)]
  val_dataloader = [[] for i in range(10)]
  test_dataloader = [[] for i in range(10)]

  for i in range(10):
    train_data = CIFAR100("dataset", 
                          train=True, 
                          transform=train_transform, 
                          download=(run+i==0),
                          random_state=RANDOM_SEED[run])
    test_data = CIFAR100("dataset", 
                         train=False, 
                         transform=test_transform, 
                         download=False,
                         random_state=RANDOM_SEED[run])
    
    train_data.set_index_map(train_data.splits[i])
    test_data.set_index_map([test_data.splits[j] for j in range(0, i+1)])
    
    train_indices, val_indices = train_data.train_val_split(VAL_SIZE, RANDOM_SEED[run])
    
    train_subset[i] = copy(Subset(train_data, train_indices))
    val_subset = Subset(train_data, val_indices)

    tmp_dl = DataLoader(val_subset,
                       batch_size=BATCH_SIZE,
                       shuffle=True, 
                       num_workers=4,
                       drop_last=True)
    val_dataloader[i] = copy(tmp_dl)

    tmp_dl = DataLoader(test_data,
                       batch_size=BATCH_SIZE,
                       shuffle=True, 
                       num_workers=4,
                       drop_last=True)
    test_dataloader[i] = copy(tmp_dl)

  return train_dataloader, val_dataloader, test_dataloader, train_subset

### Going on with the model
This is the main iCaRL step.

This step is run 3 times with different `RANDOM_SEED`.
Here the model is instantiated, trained and tested.

Results and some statistics are then stored in the variable `logs`.

In [ ]:
logs = [[] for i in range(len(RANDOM_SEED))]
best_net_tot_classes = [None for i in range(len(RANDOM_SEED))]

for run in range(len(RANDOM_SEED)):
  print("#################################")
  print(f"Radom seed: {RANDOM_SEED[run]}")
  print("")

  # get data_subsets separated in incremental groups of 10 classes
  train_dl, val_dl, test_dl, train_set = do_group_classes(run)

  #create the resnet
  net = resnet32()
  
  trainer = owrEnsemble(DEVICE,
                  net,
                  LR,
                  MOMENTUM,
                  WEIGHT_DECAY,
                  MILESTONES,
                  GAMMA,
                  train_dl,
                  val_dl,
                  test_dl,
                  BATCH_SIZE,
                  train_set,
                  train_transform,
                  test_transform,
                  TESTING_MODE,
                  REJECTION_THRESHOLD,
                  N_ESTIMATORS,
                  CONFIDENCE) 

  #train and evaluate the model
  logs[run] = trainer.train_model(NUM_EPOCHS)

  best_net_tot_classes[run] = deepcopy(trainer.best_net)

  print("#################################")
  print("")
  print("")

#################################
Radom seed: 20




Extracting dataset/cifar-100-python.tar.gz to dataset


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Log will be saved in '/content/ProjectMLDL/logs'.
Create folder 'logs/'
Start logging into file /content/ProjectMLDL/logs/classification_mnist_mlp-2021_07_12_19_31.log...
Length of exemplars set: 0


2021-07-12 19:31:06,362 - INFO: lr: 0.10000 | Epoch: 000 | Batch: 000 | Loss: 2.30382 | Correct: 17/128


...

2021-07-12 20:47:46,300 - INFO: lr: 0.00125 | Epoch: 069 | Batch: 000 | Loss: 0.21625 | Correct: 121/128


None


2021-07-12 20:48:01,869 - INFO: Save the snapshot model with index: 4
2021-07-12 20:48:02,950 - INFO: n_estimators: 5 | Validation Acc: 78.460 % | Historical Best: 79.018 %


Group 5 Finished!
Target number of exemplars: 40
Randomly extracting exemplars from class 0 of current split... Extracted 40 exemplars.
Randomly extracting exemplars from class 1 of current split... Extracted 40 exemplars.
Randomly extracting exemplars from class 2 of current split... Extracted 40 exemplars.
Randomly extracting exemplars from class 3 of current split... Extracted 40 exemplars.
Randomly extracting exemplars from class 4 of current split... Extracted 40 exemplars.
Randomly extracting exemplars from class 5 of current split... Extracted 40 exemplars.
Randomly extracting exemplars from class 6 of current split... Extracted 40 exemplars.
Randomly extracting exemplars from class 7 of current split... Extracted 40 exemplars.
Randomly extracting exemplars from class 8 of current split... Extracted 40 exemplars.
Randomly extracting exemplars from class 9 of current split... Extracted 40 exemplars.
[0.6622596153846154, 0.8623798076923077, 0.9475160256410257],[0.22455929487179488

NameError: ignored

### Store logs in more usable dtype

In [ ]:
train_loss = [[logs[run_i]['group_train_loss'][i] for i in range(5)] for run_i in range(len(RANDOM_SEED))]
train_accuracy = [[logs[run_i]['group_train_accuracies'][i] for i in range(5)] for run_i in range(len(RANDOM_SEED))]
val_loss = [[logs[run_i]['val_losses'][i] for i in range(5)] for run_i in range(len(RANDOM_SEED))]
val_accuracy = [[logs[run_i]['val_accuracies'][i] for i in range(5)] for run_i in range(len(RANDOM_SEED))]
test_accuracy = [[logs[run_i]['test_accuracies'][i] for i in range(5)] for run_i in range(len(RANDOM_SEED))]
predictions = [logs[run_i]['predictions'].cpu().data.numpy().tolist() for run_i in range(len(RANDOM_SEED))]
true_labels = [logs[run_i]['true_labels'].cpu().data.numpy().tolist() for run_i in range(len(RANDOM_SEED))]

open_values = [[logs[run_i]['open_values'][i] for i in range(5)] for run_i in range(len(RANDOM_SEED))]
closed_values = [[logs[run_i]['closed_values'][i] for i in range(5)] for run_i in range(len(RANDOM_SEED))]

### Save the model

#### Saving logs in JSON files

In [ ]:
import json

with open('states/reject_train_loss.json', 'w') as f:
  json.dump(train_loss, f)
f.close
with open('states/reject_train_accuracy.json', 'w') as f:
  json.dump(train_accuracy, f)
f.close
with open('states/reject_val_loss.json', 'w') as f:
  json.dump(val_loss, f)
f.close  
with open('states/reject_val_accuracy.json', 'w') as f:
  json.dump(val_accuracy, f)
f.close
with open('states/reject_test_accuracy.json', 'w') as f:
  json.dump(test_accuracy, f)
f.close
with open('states/reject_predictions.json', 'w') as f:
  json.dump(predictions, f)
f.close
with open('states/reject_true_labels.json', 'w') as f:
  json.dump(true_labels, f)
f.close

#### Saving best resnet on 100 classes for each seed

### Print some graphs

In [ ]:
from data.utils_plot import plot_train_val, plot_test_accuracies, plot_confusion_matrix

In [ ]:
train_loss = np.array(train_loss)
train_accuracy = np.array(train_accuracy)
val_loss = np.array(val_loss)
val_accuracy = np.array(val_accuracy)
test_accuracy = np.array(test_accuracy)



train_loss_stats = np.array([train_loss.mean(0), train_loss.std(0)]).transpose()
train_accuracy_stats = np.array([train_accuracy.mean(0), train_accuracy.std(0)]).transpose()
val_loss_stats = np.array([val_loss.mean(0), val_loss.std(0)]).transpose()
val_accuracy_stats = np.array([val_accuracy.mean(0), val_accuracy.std(0)]).transpose()
test_accuracy_stats = np.array([test_accuracy.mean(0), test_accuracy.std(0)]).transpose()


open_values = np.array(open_values)
closed_values = np.array(closed_values)

In [ ]:
threshold = 0.75
print(f"Threshold: {threshold}")
fig, axs = plt.subplots(1,2)
fig.set_size_inches(15, 5)



a,b,c = axs[0].hist(closed_values[0][4].cpu().numpy(), color = 'red', edgecolor = 'black', bins = [0,threshold,1])
below = c[0].get_height()
above = c[1].get_height()
print(f"closed_below:{below}, close_above:{above}")
print(f"closed world rejection rate: {below/(below+above):.2f} (should be 0)")


a,b,c = axs[1].hist(open_values[0][4].cpu().numpy(), color = 'blue', edgecolor = 'black', bins = [0,threshold,1])
below = c[0].get_height()
above = c[1].get_height()
print(f"open_below:{below}, open_above:{above}")
print(f"open world rejection rate: {below/(below+above):.2f} (should be 1)")

In [ ]:
mask = logs[run]['predictions'] == 50
rejected = torch.sum(mask)
total = logs[run]['predictions'].size(0)

print(f"rejection rate: {rejected/total:.2f}")




#### Train validation loss

In [ ]:
def plot_losses(train_stats, val_stats, loss = bool, save_directory = None):
  train_mean = np.array(train_stats)[:, 0]
  train_std = np.array(train_stats)[:, 1]
  val_mean = np.array(val_stats)[:, 0]
  val_std = np.array(val_stats)[:, 1]
  fig, ax = plt.subplots(figsize = (10, 5), dpi = 100)
  x = np.arange(10,51,10)
  ax.errorbar(x, train_mean, train_std, label = 'Training')
  ax.errorbar(x, val_mean, val_std, label = 'Validation')
  if loss:
    ax.set_title("Training and validation loss")
    ax.set_ylabel("Loss")
  else:
    ax.set_title("Training and validation accuracy")
    ax.set_ylabel("Accuracy")
  ax.set_xlabel("Number of classes")
  plt.tight_layout()
  ax.legend()
  if save_directory != None:
    fig.savefig(save_directory)
  plt.show()

plot_losses(train_loss_stats, val_loss_stats, loss = True)

#### Train validation accuracy

In [ ]:
def plot_acc(train_stats, val_stats, loss = bool, save_directory = None):
  train_mean = np.array(train_stats)[:, 0]
  train_std = np.array(train_stats)[:, 1]
  val_mean = np.array(val_stats)[:, 0]
  val_std = np.array(val_stats)[:, 1]
  fig, ax = plt.subplots(figsize = (10, 5), dpi = 100)
  x = np.arange(10, 51, 10)
  ax.errorbar(x, train_mean, train_std, label = 'Training')
  ax.errorbar(x, val_mean, val_std, label = 'Validation')
  if loss:
    ax.set_title("Training and validation loss")
    ax.set_ylabel("Loss")
  else:
    ax.set_title("Training and validation accuracy")
    ax.set_ylabel("Accuracy")
  ax.set_xlabel("Number of classes")
  plt.tight_layout()
  ax.legend()
  if save_directory != None:
    fig.savefig(save_directory)
  plt.show()

plot_acc(train_accuracy_stats, val_accuracy_stats, loss = False)

#### Test accuracy

In [ ]:
def plot_test(stats, save_directory = None):
  mean = np.array(stats)[:, 0]
  std = np.array(stats)[:, 1]
  fig, ax = plt.subplots(figsize = (10, 5), dpi = 100)
  x = np.arange(10, 51, 10)
  ax.errorbar(x, mean, std)
  ax.set_title("Test accuracy")
  ax.set_xlabel("Number of classes")
  ax.set_ylabel("Accuracy")
  plt.tight_layout()
  if save_directory != None:
    fig.savefig(save_directory)
  plt.show()
  
plot_test(test_accuracy_stats)

#### Confusion Matrix

In [ ]:
for run in range(len(RANDOM_SEED)):
  targets = np.array(true_labels[run])
  preds = np.array(predictions[run])

  plot_confusion_matrix(targets, preds, RANDOM_SEED[run], 'iCaRL')